In [3]:
from snowflake.snowpark import Session
# Define - connection_parameters as mentioned in creating session 
session = Session.builder.configs(connection_parameters).create()  

In [4]:
session.sql('CREATE OR REPLACE TABLE sample_product_data (id INT, parent_id INT, category_id INT, name VARCHAR, serial_number VARCHAR, key INT, "3rd" INT)').collect()

[Row(status='Table SAMPLE_PRODUCT_DATA successfully created.')]

In [5]:

session.sql("""
INSERT INTO sample_product_data VALUES
(1, 0, 5, 'Product 1', 'prod-1', 1, 10),
(2, 1, 5, 'Product 1A', 'prod-1-A', 1, 20),
(3, 1, 5, 'Product 1B', 'prod-1-B', 1, 30),
(4, 0, 10, 'Product 2', 'prod-2', 2, 40),
(5, 4, 10, 'Product 2A', 'prod-2-A', 2, 50),
(6, 4, 10, 'Product 2B', 'prod-2-B', 2, 60),
(7, 0, 20, 'Product 3', 'prod-3', 3, 70),
(8, 7, 20, 'Product 3A', 'prod-3-A', 3, 80),
(9, 7, 20, 'Product 3B', 'prod-3-B', 3, 90),
(10, 0, 50, 'Product 4', 'prod-4', 4, 100),
(11, 10, 50, 'Product 4A', 'prod-4-A', 4, 100),
(12, 10, 50, 'Product 4B', 'prod-4-B', 4, 100)
 """).collect()


[Row(number of rows inserted=12)]

In [8]:
session.sql("SELECT count(*) FROM sample_product_data").collect()

[Row(COUNT(*)=12)]

In [32]:
# create a DataFrame - from data in a table, view, or stream, call the table method
df_table = session.table("sample_product_data")

In [33]:
df_table.show()

-------------------------------------------------------------------------------------
|"ID"  |"PARENT_ID"  |"CATEGORY_ID"  |"NAME"      |"SERIAL_NUMBER"  |"KEY"  |"3rd"  |
-------------------------------------------------------------------------------------
|1     |0            |5              |Product 1   |prod-1           |1      |10     |
|2     |1            |5              |Product 1A  |prod-1-A         |1      |20     |
|3     |1            |5              |Product 1B  |prod-1-B         |1      |30     |
|4     |0            |10             |Product 2   |prod-2           |2      |40     |
|5     |4            |10             |Product 2A  |prod-2-A         |2      |50     |
|6     |4            |10             |Product 2B  |prod-2-B         |2      |60     |
|7     |0            |20             |Product 3   |prod-3           |3      |70     |
|8     |7            |20             |Product 3A  |prod-3-A         |3      |80     |
|9     |7            |20             |Product 3B  |pro

In [10]:
# Create df from specific value
df = session.create_dataframe([1,2,3,4]).to_df("a")
df.show()

-------
|"A"  |
-------
|1    |
|2    |
|3    |
|4    |
-------



In [15]:
# Create a DataFrame with 4 columns, “a”, “b”, “c” and “d”:
df = session.create_dataframe([[1,2,3,4]],schema=["a", "b", "c", "d"])
df.show()

-------------------------
|"A"  |"B"  |"C"  |"D"  |
-------------------------
|1    |2    |3    |4    |
-------------------------



In [16]:
# Create another DataFrame with 4 columns, “a”, “b”, “c” and “d”:
from snowflake.snowpark import Row
df = session.create_dataframe([Row(a=1, b=2, c=3, d=4)])
df.show()

-------------------------
|"A"  |"B"  |"C"  |"D"  |
-------------------------
|1    |2    |3    |4    |
-------------------------



In [19]:
# Create a DataFrame and specify a schema:
from snowflake.snowpark.types import IntegerType, StringType, StructType, StructField
schema = StructType(
    [
        StructField("a",IntegerType()),
        StructField("b",StringType()),
        StructField("c",StringType())
    ]
)
df = session.create_dataframe([[1,"snow","flake"],[2, "AWS","Cloud"]],schema)
df.show()

----------------------
|"A"  |"B"   |"C"    |
----------------------
|1    |snow  |flake  |
|2    |AWS   |Cloud  |
----------------------



In [23]:
# To create a DataFrame containing a range of values
df = session.range(1,10,3).to_df("a") # increment by 3
df.show()

-------
|"A"  |
-------
|1    |
|4    |
|7    |
-------



In [25]:
# To create a DataFrame to hold the data from a file in a stage, use the read property to get a DataFrameReader object. 
# In the DataFrameReader object, call the method corresponding to the format of the data in the file:
df = session.read.json("@json_input_tb/random_json_file.json")
df.show()

------------------------------------------------------
|"$1"                                                |
------------------------------------------------------
|[                                                   |
|  {                                                 |
|    "_id": "65d8f0f28b486933b8e84db9",              |
|    "about": "Id dolore id exercitation pariatu...  |
|    "address": "354 Railroad Avenue, Westmorela...  |
|    "age": 27,                                      |
|    "balance": "$2,128.87",                         |
|    "company": "FURNAFIX",                          |
|    "email": "joanneblevins@furnafix.com",          |
|    "eyeColor": "brown",                            |
|    "favoriteFruit": "strawberry",                  |
|    "friends": [                                    |
|      {                                             |
|        "id": 0,                                    |
|        "name": "Beck Cantrell"                     |
|      }, 

In [30]:
#Set schema is mandatory
schema = StructType(
    [
        StructField("a",StringType()),
        StructField("b", StringType())
    ]
)
df = session.read.schema(schema).csv("@sample_data_stage/")
df.show()

---------------
|"A"   |"B"   |
---------------
|Rank  |CCA3  |
|36    |AFG   |
|138   |ALB   |
|34    |DZA   |
|213   |ASM   |
|203   |AND   |
|42    |AGO   |
|224   |AIA   |
|201   |ATG   |
|33    |ARG   |
---------------



In [31]:
# Create a DataFrame from a SQL query
df = session.sql("select name from sample_product_data")
df.show()

--------------
|"NAME"      |
--------------
|Product 1   |
|Product 1A  |
|Product 1B  |
|Product 2   |
|Product 2A  |
|Product 2B  |
|Product 3   |
|Product 3A  |
|Product 3B  |
|Product 4   |
--------------



In [ ]:
#I t is possible to use the sql method to execute SELECT statements that retrieve data from tables and staged files. 
# However using the table method and read property offer better syntax highlighting, error highlighting.

In [35]:
session.close()